In [32]:
import numpy as np

In [33]:
rand = lambda n: np.abs(np.random.rand(n)*10)
o2sat = rand(10)*10
o2 = rand(10)
iage = rand(10)
aou = o2sat - o2
our = aou/iage
print o2sat
print o2
print aou
print iage

[ 13.74619919  42.23612198  72.20190223  13.87003704  93.08994944
  39.66189759  42.11751004  73.53975118  53.04119637  30.2663611 ]
[ 5.57386209  6.13933142  5.17240958  2.64894555  1.58139145  7.07991298
  1.34095879  6.4679584   3.28590974  6.12963958]
[  8.17233711  36.09679056  67.02949265  11.2210915   91.50855799
  32.58198461  40.77655126  67.07179278  49.75528663  24.13672152]
[ 9.92313776  7.3392318   3.8937805   3.0149675   0.41592011  0.22904358
  9.75628831  7.91147492  7.34352249  0.07895635]


In [34]:
print aou.mean()

42.8350606615


In [35]:
print o2sat.mean() - o2.mean()

42.8350606615


In [36]:
print our

[   0.82356381    4.91833363   17.21450212    3.72179518  220.01474889
  142.25233673    4.17951479    8.47778618    6.77539787  305.69704479]


In [37]:
print aou
print aou.mean()

[  8.17233711  36.09679056  67.02949265  11.2210915   91.50855799
  32.58198461  40.77655126  67.07179278  49.75528663  24.13672152]
42.8350606615


In [38]:
print our * iage

[  8.17233711  36.09679056  67.02949265  11.2210915   91.50855799
  32.58198461  40.77655126  67.07179278  49.75528663  24.13672152]


In [39]:
print our.mean()*iage.mean()

356.368590163
